<a href="https://colab.research.google.com/github/aroonalok/deep-learning-foundations/blob/main/Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import requests
from google.colab import files
import pandas as pd
import pytz
from datetime import datetime
import re
import openpyxl
from openpyxl.styles import Font, Color, Border, Side

Config

In [48]:
# @title Date fields
FROM_DATE = '2024-04-02' # @param {type:"date"}
TO_DATE = '2024-04-02' # @param {type:"date"}
DOWNLOAD_REPORT = False # @param {type:"boolean"}

DATE_FORMAT = '%Y-%m-%d'
if datetime.strptime(FROM_DATE, DATE_FORMAT) > datetime.strptime(TO_DATE, DATE_FORMAT):
  raise AssertionError("TO_DATE should be greater than FROM_DATE ")

In [49]:
ORGANIZATION_ID = 60027298359
CODE = "1000.55d67894e14d35a5ae4ef3ba694c8982.a724e7d2c7b37639d4a3aac2eea554da"
CLIENT_ID = "1000.VN8R67MXBXHMVQVD7RPH6QLBGDMWAD"
CLIENT_SECRET = "7829df77f55363a44d97c30e7d1a13e3504d9faccc"
SCOPE = "ZohoBooks.fullaccess.all"

MAX_RECORDS_PER_PAGE = 1000
IST_TIMEZONE = pytz.timezone('Asia/Kolkata')

API_CALLS_COUNTER = 0

In [50]:
ROOT_API_ENDPOINT = "https://www.zohoapis.in/books/v3/"

In [51]:
def formatTimestamp(timestamp):
  timestamp_datetime = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S%z')
  ist_datetime = timestamp_datetime.astimezone(IST_TIMEZONE)
  return ist_datetime.strftime("%d-%m-%Y %H:%M")

def getItemType(item):
  return "Book" if item['hsn_or_sac'] == '49011010' else "Article"

OAuth

In [52]:
OAUTH_ROOT_URL = "https://accounts.zoho.in/oauth/v2/token?"
OAUTH_REDIRECT_URI = "http://www.zoho.in/books"

def getOauthToken(code, client_id, client_secret):
  global API_CALLS_COUNTER
  params = []
  params.append("code={CODE}".format(CODE=code))
  params.append("client_id={CLIENT_ID}".format(CLIENT_ID=client_id))
  params.append("client_secret={CLIENT_SECRET}".format(CLIENT_SECRET=client_secret))
  params.append("redirect_uri={REDIRECT_URI}".format(REDIRECT_URI=OAUTH_REDIRECT_URI))
  params.append("grant_type=authorization_code")
  url = OAUTH_ROOT_URL+"&".join(params)
  payload = {}
  headers = {}
  response = requests.request("POST", url, headers=headers, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()

def renewOauthToken(refresh_token, client_id, client_secret):
  global API_CALLS_COUNTER
  params = []
  params.append("refresh_token={REFRESH_TOKEN}".format(REFRESH_TOKEN=refresh_token))
  params.append("client_id={CLIENT_ID}".format(CLIENT_ID=client_id))
  params.append("client_secret={CLIENT_SECRET}".format(CLIENT_SECRET=client_secret))
  params.append("redirect_uri={REDIRECT_URI}".format(REDIRECT_URI=OAUTH_REDIRECT_URI))
  params.append("grant_type=refresh_token")
  url = OAUTH_ROOT_URL+"&".join(params)
  payload = {}
  headers = {}
  response = requests.request("POST", url, headers=headers, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()

def saveOauthToken(oauth_token, filename='authorization_token.txt'):
  with open(filename, 'w') as token_file:
    token_file.write(oauth_token['refresh_token'])
    files.download(filename)

def createSession(access_token):
  session = requests.Session()
  session.headers.update({'Authorization': 'Zoho-oauthtoken {access_token}'.format(access_token=access_token)})
  return session

DO NOT RUN THIS CELL

In [53]:
#oauth_token = getOauthToken(CODE,CLIENT_ID,CLIENT_SECRET)
#saveOauthToken(oauth_token)

Organization

In [54]:
def getOrganizationInfo(session):
  global API_CALLS_COUNTER
  url = ROOT_API_ENDPOINT+'organizations'
  payload = {}
  response = session.get(url,data=payload)
  API_CALLS_COUNTER += 1
  return response.json()

Renew Access Token

In [55]:
oauth_token = renewOauthToken('1000.8e82d21986ea040cf450df7a7afee4d8.84fa32b03a3316dbbdcba8033e4fc3ed',CLIENT_ID, CLIENT_SECRET)
session = createSession(oauth_token['access_token'])

In [56]:
getOrganizationInfo(session)

{'code': 0,
 'message': 'success',
 'organizations': [{'isOrgNotSupported': False,
   'organization_id': '60027298359',
   'name': 'Ramakrishna Math Halasuru',
   'contact_name': 'Ramakrishna Math Halasuru',
   'email': 'halasuru.bookstall@rkmm.org',
   'source': 2,
   'country': 'India',
   'country_code': 'IN',
   'org_settings': True,
   'is_ziedition': False,
   'custom_field_type': 2,
   'custom_fields': [{'index': 1, 'label': '', 'value': ''},
    {'index': 2, 'label': '', 'value': ''},
    {'index': 3, 'label': '', 'value': ''},
    {'index': 4, 'label': '', 'value': ''},
    {'index': 5, 'label': '', 'value': ''}],
   'is_sku_enabled': True,
   'phone': '6364083518',
   'org_type': 'live',
   'state_code': 'KA',
   'state': 'Karnataka',
   'zoho_one_org': '',
   'zi_zb_edition': 1,
   'org_created_app_source': 12,
   'zi_zb_client': 1,
   'is_solo_org': False,
   'partners_domain': '',
   'version': 'india',
   'version_formatted': 'India',
   'is_search360_enabled': 'true',
  

Invoices

In [57]:
def getInvoices(session, from_date=None, to_date=None, additional_params=[]):
  global API_CALLS_COUNTER
  params = additional_params
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  params.append("per_page={MAX_RECORDS_PER_PAGE}".format(MAX_RECORDS_PER_PAGE=MAX_RECORDS_PER_PAGE))
  params.append("sort_column=created_time")
  if from_date is not None:
    params.append("date_start={FROM_DATE}".format(FROM_DATE=from_date))
  if to_date is not None:
    params.append("date_end={TO_DATE}".format(TO_DATE=to_date))
  url = ROOT_API_ENDPOINT+"invoices?"+"&".join(params)

  results = []
  payload = {}
  send_request = True
  current_page = 1
  while send_request:
    response = session.get(url+"&page={PAGE}".format(PAGE=current_page),data=payload)
    API_CALLS_COUNTER += 1
    #perform exception handling here
    results += response.json()['invoices']
    send_request = response.json()['page_context']['has_more_page']
    current_page += 1

  return pd.DataFrame.from_records(results) if results else None

def getInvoiceDetails(session, invoice_id):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  url = ROOT_API_ENDPOINT+"invoices/{invoice_id}?".format(invoice_id=invoice_id)+"&".join(params)
  payload = {}
  response = session.get(url, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()
  #perform exception handling here

def getPaymentsForInvoice(session, invoice_id):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  url = ROOT_API_ENDPOINT+"invoices/{invoice_id}/payments?".format(invoice_id=invoice_id)+"&".join(params)
  payload = {}
  response = session.get(url, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()
  #perform exception handling here

In [58]:
#invoices = getInvoices(session, from_date='2024-03-21')

In [59]:
#invoices

Payments

In [60]:
def getPayments(session, from_date=None, to_date=None):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  params.append("per_page={MAX_RECORDS_PER_PAGE}".format(MAX_RECORDS_PER_PAGE=MAX_RECORDS_PER_PAGE))
  params.append("sort_column=created_time")
  if from_date is not None:
    params.append("date_start={FROM_DATE}".format(FROM_DATE=from_date))
  if to_date is not None:
    params.append("date_end={TO_DATE}".format(TO_DATE=to_date))
  url = ROOT_API_ENDPOINT+"customerpayments?"+"&".join(params)

  results = []
  payload = {}
  send_request = True
  current_page = 1
  while send_request:
    response = session.get(url+"&page={PAGE}".format(PAGE=current_page), data=payload)
    API_CALLS_COUNTER += 1
    #perform exception handling here
    results += response.json()['customerpayments']
    send_request = response.json()['page_context']['has_more_page']
    current_page += 1

  return pd.DataFrame.from_records(results) if results else None

# Daily Sales Summary Report

**Policies**

 - For a given Invoice, there can be only one payment mode. For example, an invoice cannot have partial payments via Card(s) and Cash / UPI.
 - All Credit settlement is done fully and at once. No installments. As in the above case, there can not be partial credit and partial payment for an Invoice.

In [61]:
PAYMENT_MODE_CREDIT = "Credit"
DAILY_SALES_REPORT_CAPTION = "DAILY SALES SUMMARY REPORT FROM: {from_date} TO: {to_date}"
SALES_TYPE_REPORT_CAPTION = "SALES TYPE SUMMARY REPORT FROM: {from_date} TO: {to_date}"
INTRA_STATE_TAX_REPORT_CAPTION = "INTRA STATE TAX REPORT FROM: {from_date} TO: {to_date}"
TOTAL_INTRA_STATE_TAX_REPORT_CAPTION = "INTRA STATE TAX SUMMARY REPORT FROM: {from_date} TO: {to_date}"
FILENAME = "report-from-{from_date}-to-{to_date}.xlsx"
SALES_SHEET_NAME = "Sales"
SALES_TYPE_SHEET_NAME = "Sales Type"
GST_SHEET_NAME = "GST"
TOTAL_GST_SHEET_NAME = "TOTAL GST"
NO_DATA_MESSAGE = "NO DATA PRESENT FROM: {from_date} TO: {to_date}"

In [62]:
def createDataForReporting(session, from_date=None, to_date=None):
  invoices = getInvoices(session, from_date, to_date)
  if invoices is None:
    return None, None, None, None

  invoice_data_rows = []
  sales_type_data_rows = []
  intra_state_tax_data_rows = []
  for invoice_id in invoices['invoice_id']:
    invoice = getInvoiceDetails(session, invoice_id)['invoice']
    payments = getPaymentsForInvoice(session, invoice_id)['payments']
    #assert len(payments) <= 1
    payment_mode = payments[0]['payment_mode'] if payments else PAYMENT_MODE_CREDIT
    invoice_data_rows.append({'Invoice No.' : invoice['invoice_number'],
                              'Time' : formatTimestamp(invoice['created_time']),
                              'Payment Mode' : payment_mode,
                              'Amount (including tax)' : invoice['bcy_total']})
    for item in invoice['line_items']:
      item_total_inclusive_of_tax = item['rate']*item['quantity']
      sales_type_data_rows.append({'Item Type' : getItemType(item),
                                   'Amount (including GST)' : item_total_inclusive_of_tax,
                                   'Payment Mode' : payment_mode})
      cgst_amount, cgst_percent, sgst_amount, sgst_percent = 0.0, '0%', 0.0, '0%'
      for tax in item['line_item_taxes']:
        if tax['tax_name'].startswith("CGST"):
          cgst_amount, cgst_percent = tax['tax_amount'], re.search('\d*%', tax['tax_name']).group()
        elif tax['tax_name'].startswith("SGST"):
          sgst_amount, sgst_percent = tax['tax_amount'], re.search('\d*%', tax['tax_name']).group()
      intra_state_tax_data_rows.append({'Date' : invoice['date'],
                                        'SKU' : item['sku'],
                                        'Amount (excluding GST)' : item['item_total'],
                                        'CGST Amount' : cgst_amount,
                                        'SGST Amount' : sgst_amount,
                                        'CGST Percent' : cgst_percent,
                                        'SGST Percent' : sgst_percent,
                                        'Amount (including GST)' : item_total_inclusive_of_tax})

  invoice_df = pd.DataFrame(data = invoice_data_rows)
  sales_type_df = pd.DataFrame(data = sales_type_data_rows).groupby(['Item Type', 'Payment Mode']).sum()
  intra_state_tax_df = pd.DataFrame(data = intra_state_tax_data_rows)
  total_intra_state_tax_df = intra_state_tax_df[['Amount (excluding GST)','CGST Amount','SGST Amount','Amount (including GST)']].sum().to_frame(name="Total")
  return invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df

def addCaption(df, caption):
  df.columns = pd.MultiIndex.from_product([[caption], df.columns])
  return df

def set_borders_and_save(filename):
  thin = Side(border_style="thin", color="000000")
  wb = openpyxl.load_workbook(filename)
  for sheet in wb.sheetnames:
    ws = wb[sheet]
    for row in ws[ws.dimensions]:
      for cell in row:
        cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
  wb.save(filename)

def generateReport(from_date=None, to_date=None):
  oauth_token = renewOauthToken('1000.8e82d21986ea040cf450df7a7afee4d8.84fa32b03a3316dbbdcba8033e4fc3ed',CLIENT_ID, CLIENT_SECRET)
  session = createSession(oauth_token['access_token'])

  report_data = createDataForReporting(session, from_date, to_date)

  if all(df is None for df in report_data):
    return (NO_DATA_MESSAGE.format(from_date=from_date, to_date=to_date),)*len(report_data)

  invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df = report_data
  invoice_df = addCaption(invoice_df, DAILY_SALES_REPORT_CAPTION.format(from_date=from_date, to_date=to_date))
  sales_type_df = addCaption(sales_type_df, SALES_TYPE_REPORT_CAPTION.format(from_date=from_date, to_date=to_date))
  intra_state_tax_df = addCaption(intra_state_tax_df, INTRA_STATE_TAX_REPORT_CAPTION.format(from_date=from_date, to_date=to_date))
  total_intra_state_tax_df = addCaption(total_intra_state_tax_df, TOTAL_INTRA_STATE_TAX_REPORT_CAPTION.format(from_date=from_date, to_date=to_date))

  filename = FILENAME.format(from_date=from_date, to_date=to_date)
  with pd.ExcelWriter(filename) as writer:
    invoice_df.to_excel(writer, sheet_name=SALES_SHEET_NAME, index=True)
    sales_type_df.to_excel(writer, sheet_name=SALES_TYPE_SHEET_NAME, index=True)
    intra_state_tax_df.to_excel(writer, sheet_name=GST_SHEET_NAME, index=True)
    total_intra_state_tax_df.to_excel(writer, sheet_name=TOTAL_GST_SHEET_NAME, index=True)

  set_borders_and_save(filename)
  if DOWNLOAD_REPORT:
    files.download(filename)

  return invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df




In [63]:
invoice_df, sales_type_df, intra_state_tax_df, total_intra_state_tax_df = generateReport(from_date=FROM_DATE, to_date=TO_DATE)

In [64]:
invoice_df

'NO DATA PRESENT FROM: 2024-04-02 TO: 2024-04-02'

In [65]:
sales_type_df

'NO DATA PRESENT FROM: 2024-04-02 TO: 2024-04-02'

In [66]:
intra_state_tax_df

'NO DATA PRESENT FROM: 2024-04-02 TO: 2024-04-02'

In [67]:
total_intra_state_tax_df

'NO DATA PRESENT FROM: 2024-04-02 TO: 2024-04-02'

In [68]:
API_CALLS_COUNTER

4